<a href="https://colab.research.google.com/github/baranouskiart/rmf_analysis/blob/main/product_amazon_sellers_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RMF ANALYSIS - SELLERS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
import pandas as pd
import re
# DATASET: https://docs.google.com/spreadsheets/d/1SnXmSfOFwXgOqOTr_DHlILvkKPKfxTdHhRjyizuTtng/edit?usp=sharing
#There is no transaction data, like invoice and date. However I could treat each review as a “purchase”,
#and calculate the recency, frequency, and “monetary” value (perhaps based on rating or price) of reviews for each product.
#This would give you an idea of which products are being reviewed most often and most favorably.

#Load Data

In [2]:
df = pd.read_csv('https://query.data.world/s/2e4tqfh2w4vfgje7i6b5xg666jmk5w?dws=00000') #uniq_id, price, number_available_in_stock [sep]
df.head() #amazon co-ecommerce sample (not full dataset)

,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


In [4]:
# Define a function to remove symbols and convert to float
def convert_to_float(x):
  try:
    # Remove leading/trailing characters except numbers, ".", "+" or "-".
    return float(re.sub(r"[^\d\.]", "", x))
  except:
    # Handle potential errors (e.g., empty string) and return missing value (e.g., np.nan)
    return pd.NA

# Apply the function to 'price' and 'number_available_in_stock' columns
df['price'] = df['price'].apply(convert_to_float)
df['number_available_in_stock'] = df['number_available_in_stock'].apply(convert_to_float)
df = df.rename(columns={'price': 'price_£'})
df = df.rename(columns={'number_available_in_stock': 'new_in_stock'})
# Print the modified DataFrame
df.head()

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,3.42,5.0,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,16.99,<NA>,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,9.99,2.0,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,39.99,<NA>,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,32.19,<NA>,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


## Data Cleaning

In [5]:
df['Total']=df['new_in_stock']*df['price_£']

In [6]:
df['Total'].head()

0     17.1
1     <NA>
2    19.98
3     <NA>
4     <NA>
Name: Total, dtype: object

In [7]:
# Assuming df is your DataFrame, it could be the trasactional information abot purchase but there is no URL here
cols_to_convert = ['number_of_reviews']

for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
df.head()

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,3.42,5.0,15.0,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ...",17.1
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,16.99,<NA>,2.0,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ...",<NA>
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,9.99,2.0,17.0,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel...",19.98
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,39.99,<NA>,1.0,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN,<NA>
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,32.19,<NA>,3.0,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN,<NA>


In [9]:
df.info()  # column SELLERS need to be rename like sellers_prices

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      10000 non-null  object 
 1   product_name                                 10000 non-null  object 
 2   manufacturer                                 9993 non-null   object 
 3   price_£                                      8547 non-null   object 
 4   new_in_stock                                 7500 non-null   object 
 5   number_of_reviews                            9980 non-null   float64
 6   number_of_answered_questions                 9235 non-null   float64
 7   average_review_rating                        9982 non-null   object 
 8   amazon_category_and_sub_category             9310 non-null   object 
 9   customers_who_bought_this_item_also_bought   8938 non-null   object 
 10 

In [10]:
df.isnull().sum() #There is no difference in descriptions
#BIG NUMBER OF ZERO-VALUES: sellers, number_available_in_stock, price, customer_questions_and_answers, items_customers_buy_after_viewing_this_item

uniq_id                                           0
product_name                                      0
manufacturer                                      7
price_£                                        1453
new_in_stock                                   2500
number_of_reviews                                20
number_of_answered_questions                    765
average_review_rating                            18
amazon_category_and_sub_category                690
customers_who_bought_this_item_also_bought     1062
description                                     651
product_information                              58
product_description                             651
items_customers_buy_after_viewing_this_item    3065
customer_questions_and_answers                 9086
customer_reviews                                 21
sellers                                        3082
Total                                          3664
dtype: int64

In [11]:
df.isnull().sum()*100/len(df)
#what is percent of zero values

uniq_id                                         0.00
product_name                                    0.00
manufacturer                                    0.07
price_£                                        14.53
new_in_stock                                   25.00
number_of_reviews                               0.20
number_of_answered_questions                    7.65
average_review_rating                           0.18
amazon_category_and_sub_category                6.90
customers_who_bought_this_item_also_bought     10.62
description                                     6.51
product_information                             0.58
product_description                             6.51
items_customers_buy_after_viewing_this_item    30.65
customer_questions_and_answers                 90.86
customer_reviews                                0.21
sellers                                        30.82
Total                                          36.64
dtype: float64

In [12]:
df.loc[(df['Total'].isnull())&(df['customer_reviews'].isnull())] #how much data is not prepared for RMF (no transactions, no date)

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
1012,9082ef74a6613b19e743e5bcde529624,Teenage Mutant Ninja Turtles Mutations Mutatin...,Teenage Mutant Ninja Turtles,<NA>,<NA>,NaN,NaN,NaN,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Teenage-Mutant-Ninja-T...,"Product Description Leo, Donnie, Raph and Mike...",Technical Details Item Weight259 g Product Dim...,"Product Description Leo, Donnie, Raph and Mike...",http://www.amazon.co.uk/Teenage-Mutant-Ninja-T...,NaN,NaN,NaN,<NA>
2273,8b44622d50e0d3b82b1fe52e880f4cf8,Tamiya #74001 Side Cutter for Plastic - MK801,Tamiya,<NA>,<NA>,NaN,3.0,NaN,NaN,http://www.amazon.co.uk/TAMIYA-Tools-Cutter-Pl...,NaN,Technical Details Item Weight68 g Product Dime...,NaN,NaN,NaN,NaN,NaN,<NA>
3020,a80fc72f3abd79455ce1ec35387e9f46,"1st Birthday Assorted Multi-Coloured 12"" Latex...",Good Deals Online,2.1,<NA>,1.0,2.0,5.0 out of 5 stars,Party Supplies > Decorations > Balloons,http://www.amazon.co.uk/Birthday-Assorted-Mult...,"Colour Name:3 Packs Colours included - Blue, O...",Technical Details Manufacturer recommended age...,"Colour Name:3 Packs Colours included - Blue, O...",NaN,NaN,NaN,"{""seller""=>{""Seller_name_1""=>""Good Deals Onlin...",<NA>
3299,e1f33659de24242fb914231384eb51b6,Paper Magic Street Fighter M. Bison Adult Cost...,Street Fighter,87.99,<NA>,1.0,1.0,4.0 out of 5 stars,Fancy Dress > Costumes > Adults,http://www.amazon.co.uk/Smiffys-Akuma-Costume-...,Size Name:46-48 | Colour Name:Red *Jacket wi...,Technical Details Item Weight680 g Product Dim...,Size Name:46-48 | Colour Name:Red *Jacket wi...,NaN,NaN,NaN,"{""seller""=>{""Seller_name_1""=>""YUK"", ""Seller_pr...",<NA>
3606,58bc8b69c859e49d6162bc2570f6d340,Tamiya 35252 Model Vehicle Tiger II Königstige...,Dickie-Tamiya,126.95,<NA>,NaN,1.0,NaN,NaN,http://www.amazon.co.uk/German-King-Tiger-Arde...,1/35 Scale. Skill level 2. All Tamiya Kits are...,Technical Details Item Weight540 g Product Dim...,1/35 Scale. Skill level 2. All Tamiya Kits are...,http://www.amazon.co.uk/German-King-Tiger-Arde...,NaN,NaN,"{""seller""=>{""Seller_name_1""=>""Japan_Hobby_Shop...",<NA>
7133,5a39514399ddf4dd19a4e90ec2f5484f,Marvel Super Hero Mashers Marvel's Spider-Man ...,Marvel,<NA>,5.0,NaN,2.0,NaN,Characters & Brands > Hasbro,http://www.amazon.co.uk/Marvel-Spiderman-Mashe...,Marvel Superhero Mashers action figure of Spid...,Technical Details Item Weight113 g Product Dim...,Marvel Superhero Mashers action figure of Spid...,http://www.amazon.co.uk/Marvel-Spiderman-Mashe...,NaN,NaN,"{""seller""=>[{""Seller_name_1""=>""MANDROPS AG"", ""...",<NA>
8954,4b6464bda1c36e7befc8fef39b4f6585,Teenage Mutant Ninja Turtles Green Hooded Onesie,Teenage Mutant Ninja Turtles,<NA>,<NA>,1.0,1.0,5.0 out of 5 stars,Figures & Playsets > Science Fiction & Fantasy,http://www.amazon.co.uk/Teenage-Mutant-Ninja-T...,Size Name:7 - 8 Years Teenage Mutant Ninja Tur...,Technical Details Manufacturer recommended age...,Size Name:7 - 8 Years Teenage Mutant Ninja Tur...,NaN,NaN,NaN,"{""seller""=>{""Seller_name_1""=>""This Is It Famou...",<NA>


## Data Preprocessing

In [13]:
df.drop('average_review_rating', axis=1).describe().T

,count,mean,std,min,25%,50%,75%,max
number_of_reviews,9980.0,8.897395,28.942848,1.0,1.0,2.0,6.0,802.0
number_of_answered_questions,9235.0,1.834976,2.517268,1.0,1.0,1.0,2.0,39.0


In [14]:
df.columns

Index(['uniq_id', 'product_name', 'manufacturer', 'price_£', 'new_in_stock',
       'number_of_reviews', 'number_of_answered_questions',
       'average_review_rating', 'amazon_category_and_sub_category',
       'customers_who_bought_this_item_also_bought', 'description',
       'product_information', 'product_description',
       'items_customers_buy_after_viewing_this_item',
       'customer_questions_and_answers', 'customer_reviews', 'sellers',
       'Total'],
      dtype='object')

In [15]:
duplicates = df['product_name'].duplicated()
duplicates.sum()

36

In [16]:
duplicates = df['manufacturer'].duplicated()
duplicates.sum()

7348

In [17]:
df.groupby(['product_name'])['Total'].sum().sort_values()

product_name
"5 daisy" spacer bead 8mm x 3mm. Pack of 15                                                      0
MONSTER HIGH SKULL LIFE STICKER COLLECTION PANINI ~ 10 PACKS                                     0
MICRO SLOTCAR FIREBIRD                                                                           0
MG Metro 6R4, No.23, Repco, Rallye Great Britain, 1986, Model Car, Ready-made, IXO 1:43          0
MERRY CHRISTMAS PLAQUE RED DECORATION PACK OF 3                                                  0
                                                                                            ...   
Parrot Bebop Drone 2 (White)                                                               4839.45
Bowen Designs - Marvel Statue Thor Classic Action 49 cm                                    4879.84
Hot Toys Movie Masterpiece - Avengers Age of Ultron: Thor                                  5206.95
LEGO Star Wars 75103: First Order Transporter                                              5519.

In [18]:
df.groupby(['manufacturer'])['Total'].sum().sort_values()

manufacturer
MBNF                        0
SORA                        0
SOPHZZZZ TOY SHOP           0
SMER                        0
Fine Horse                  0
                       ...   
Bruder                23754.9
Hot Toys             26060.38
Star Wars            28490.28
Playmobil            49721.78
LEGO                  88777.0
Name: Total, Length: 2651, dtype: object

In [19]:
# sum the 'number_of_reviews', sort the values, and filter out the zeros
df_grouped = df.groupby(['manufacturer'])['number_of_reviews'].sum().sort_values()
df_grouped = df_grouped[df_grouped != 0]

In [20]:
df_grouped.tail() #most popular manufacturers (not show the tone of review)

manufacturer
Nollmit         1566.0
Star Wars       1595.0
Orchard Toys    1608.0
LEGO            2930.0
Playmobil       3222.0
Name: number_of_reviews, dtype: float64

In [21]:
df.tail()

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
9995,44d6967f083825a5de36ad4865a65bcd,Batman 1966 TV Series Action Figures - The Rid...,Mattel,22.95,5.0,3.0,3.0,5.0 out of 5 stars,Hobbies > Collectible Figures & Memorabilia > ...,http://www.amazon.co.uk/Batman-1966-Series-Act...,DC 66 Batman Classic TV Series 6 Inch Riddler ...,Technical Details Item Weight136 g Product Dim...,DC 66 Batman Classic TV Series 6 Inch Riddler ...,NaN,NaN,Realistic // 5.0 // 31 Mar. 2014 // By\n \n...,"{""seller""=>[{""Seller_name_1""=>""Star Action Fig...",114.75
9996,08f0747b6fc6687215ffb994c3a6fb32,"Star Wars Costume, Kids Stormtrooper Costume S...",Star Wars,39.99,<NA>,1.0,3.0,4.0 out of 5 stars,Characters & Brands > Star Wars > Toys,http://www.amazon.co.uk/Storm-Trooper-Costume-...,NaN,Technical Details Additional Information AS...,NaN,http://www.amazon.co.uk/Storm-Trooper-Costume-...,NaN,... what I see my grandson us going to have fu...,NaN,<NA>
9997,bf6cc073f8f24e6e338190fa16f6ee9d,Defiance Lawkeeper Metal Badge Prop Replica,Olde Scotland Yard Ltd.,43.99,3.0,1.0,3.0,5.0 out of 5 stars,Novelty & Special Use > Novelty > Accessories ...,NaN,*Includes 1x Badge with holder and chain *High...,Technical Details Item Weight159 g Product Dim...,*Includes 1x Badge with holder and chain *High...,http://www.amazon.co.uk/Olde-Scotland-Yard-Ltd...,NaN,Five Stars // 5.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""YUK"", ""Seller_p...",131.97
9998,cd783d0b8b44e631b9788b203eaaefae,Justice League of America Series 3 Green Lante...,DC Comics,49.81,3.0,1.0,3.0,5.0 out of 5 stars,Hobbies > Collectible Figures & Memorabilia > ...,NaN,Designed by Ed BenesIt's here - the third seri...,Technical Details Item Weight181 g Product Dim...,Designed by Ed BenesIt's here - the third seri...,NaN,NaN,The best sculpt in a while // 5.0 // 13 May 20...,"{""seller""=>[{""Seller_name_1""=>""Smaller World F...",149.43
9999,94d18e86e504bee1e392df7e5738b18a,Star Wars 1/72 Y-Wing Starfighter,Bandai,21.2,31.0,11.0,3.0,4.5 out of 5 stars,Characters & Brands > Star Wars > Toys,http://www.amazon.co.uk/Bandai-Star-Wars-Fight...,NaN,Technical Details Item Weight299 g Product Dim...,NaN,NaN,NaN,Gold leader // 5.0 // 31 Aug. 2015 // By\n ...,NaN,657.2


In [22]:
# sum the 'number_of_reviews', sort the values, and filter out the zeros
df_grouped = df.groupby(['manufacturer'])['number_of_answered_questions'].sum().sort_values()
df_grouped = df_grouped[df_grouped != 0]

In [23]:
df_grouped.tail() #most active by response manufacturers or sellers (not show the tone of review)

manufacturer
Mattel            203.0
Oxford Diecast    229.0
Disney            305.0
Playmobil         353.0
LEGO              390.0
Name: number_of_answered_questions, dtype: float64

In [24]:
df['manufacturer'].value_counts().head(20) #most popular manufactures in dataset

manufacturer
LEGO                             171
Disney                           167
Oxford Diecast                   156
Playmobil                        147
Star Wars                        120
Mattel                           114
Hasbro                           110
The Puppet Company               109
MyTinyWorld                       93
Corgi                             90
Hornby                            87
Scalextric                        76
Pokémon                           69
Schleich                          68
Amscan                            65
Every-occasion-party-supplies     62
FunKo                             59
Melissa & Doug                    59
Papo                              58
Tamiya                            57
Name: count, dtype: int64

## Explore the Manufacturer

In [25]:
df_Mattel=df.loc[df['manufacturer']=='Mattel'] #the most active manufacturer dataset

In [26]:
df_Mattel

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
86,4b15acfc7dc8ff73020661531b9c3a3d,Thomas and Friends Track Master Dockside Deliv...,Mattel,13.87,6.0,1.0,1.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Thomas-Friends-Trackma...,Trackmaster Revolution Dockside Delivery Crane...,Technical Details Item Weight141 g Product Dim...,Trackmaster Revolution Dockside Delivery Crane...,http://www.amazon.co.uk/Thomas-Friends-Trackma...,NaN,Five Stars // 5.0 // 17 Dec. 2015 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""JemToys"", ""Sell...",83.22
713,136d1e399d3be623ae941b82d5b083e2,Monster High advents calendar,Mattel,30.18,<NA>,2.0,3.0,4.5 out of 5 stars,NaN,http://www.amazon.co.uk/Monster-High-Advent-Be...,Monster High advents calendar - Monster High 9...,Technical Details Item Weight522 g Product Dim...,Monster High advents calendar - Monster High 9...,http://www.amazon.co.uk/Monster-High-Advent-Be...,NaN,Quite a few windows empty !!!! // 4.0 // 13 De...,"{""seller""=>{""Seller_name_1""=>""Son Services gGm...",<NA>
731,555c900756b846482ed5821ee37e412d,Mattel BCK27 - Angry Birds Advent Calendar,Mattel,29.0,3.0,1.0,6.0,4.0 out of 5 stars,NaN,http://www.amazon.co.uk/Angry-Birds-Pig-Rock-R...,Angry Birds Advent Calendar: This is a premium...,Technical Details Item Weight544 g Product Dim...,Angry Birds Advent Calendar: This is a premium...,NaN,NaN,Four Stars // 4.0 // 11 Oct. 2015 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""AT-STOCK"", ""Sel...",87.0
797,98c21ade2223060290f581d43c0ae209,Matchbox Cars - 60th Anniversary Collection - ...,Mattel,6.94,<NA>,1.0,11.0,5.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,http://www.amazon.co.uk/Matchbox-N5424-20-Car-...,NaN,Technical Details Manufacturer recommended age...,NaN,http://www.amazon.co.uk/Siku-1404-US-Patrol-Ca...,NaN,Five Stars // 5.0 // 19 Jan. 2015 // By\n \...,NaN,<NA>
799,dc603e26ae1406347d8f0323612bda3a,Disney Pixar BDW82 Cars Jessica Giampetrol & N...,Mattel,7.99,14.0,2.0,11.0,5.0 out of 5 stars,Die-Cast & Toy Vehicles > Toy Vehicles & Acces...,http://www.amazon.co.uk/Disney-Pixar-Cars-Kimb...,Disney/Pixar Cars Collector Die-cast 2 Pack Co...,Technical Details Item Weight109 g Product Dim...,Disney/Pixar Cars Collector Die-cast 2 Pack Co...,http://www.amazon.co.uk/Disney-Pixar-Cars-Kimb...,NaN,Five Stars // 5.0 // 19 Dec. 2015 // By\n \...,NaN,111.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9898,d6eeeb53a54e7611a7b9880c3c48175e,Mattel - Scene It? Friends DVD Game,Mattel,59.99,8.0,36.0,1.0,4.6 out of 5 stars,Games > DVD Games,http://www.amazon.co.uk/Friends-Official-Trivi...,Product Description Be the first to answer Fri...,Technical Details Item Weight3 Kg Product Dime...,Product Description Be the first to answer Fri...,NaN,What region is the DVD specific to? // Region ...,The perfect game for Friends Addicts! // 4.0 /...,"{""seller""=>[{""Seller_name_1""=>""tailor10"", ""Sel...",479.92
9915,4f955d75276bc16b1ff27dd88c7dc686,FIFA Scene It? DVD Game,Mattel,1.59,19.0,5.0,1.0,4.4 out of 5 stars,Games > DVD Games,http://www.amazon.co.uk/Toys-UEFA-Champions-Le...,Product Description Major new license game for...,Technical Details Item Weight1.1 Kg Product Di...,Product Description Major new license game for...,http://www.amazon.co.uk/The-Green-Board-Game-C...,Can I buy an up to date set of cards and DVD f...,Great game // 5.0 // 29 Jun. 2013 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""Angela's Books""...",30.21
9933,7b8edfb087ede832f19249372fdbba83,Scene it? Magical Moments Disney DVD Game,Mattel,45.99,4.0,7.0,2.0,4.3 out of 5 stars,Characters & Brands > Disney > 

In [27]:
df_Mattel['new_in_stock'].value_counts().head(20)

new_in_stock
3.0     19
2.0     12
1.0     12
4.0     11
5.0     10
11.0     6
6.0      5
7.0      5
10.0     5
8.0      3
9.0      3
13.0     2
14.0     2
12.0     2
19.0     2
21.0     1
15.0     1
26.0     1
17.0     1
16.0     1
Name: count, dtype: int64

##Data Group Analysis

In [28]:
df.groupby(['average_review_rating'])[['manufacturer']].nunique()#/df.groupby(['average_review_rating'])[['manufacturer']].count()

,manufacturer
average_review_rating,
2.3 out of 5 stars,1
3.0 out of 5 stars,3
3.3 out of 5 stars,2
3.5 out of 5 stars,2
3.6 out of 5 stars,3
3.7 out of 5 stars,2
3.8 out of 5 stars,1
3.9 out of 5 stars,7
4.0 out of 5 stars,729


## Data Research

In [29]:
df.groupby(['average_review_rating'])[['product_name']].nunique()/df.groupby(['average_review_rating'])[['product_name']].count()

,product_name
average_review_rating,
2.3 out of 5 stars,1.000000
3.0 out of 5 stars,1.000000
3.3 out of 5 stars,1.000000
3.5 out of 5 stars,1.000000
3.6 out of 5 stars,1.000000
3.7 out of 5 stars,1.000000
3.8 out of 5 stars,1.000000
3.9 out of 5 stars,1.000000
4.0 out of 5 stars,0.998476


In [30]:
df.sample(20)

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
7217,2f80bb29322692d0b0c6e5f9ba428d0d,XT-XINTE 3D printer accessories nozzle cleanin...,XINTE,1.72,<NA>,1.0,1.0,5.0 out of 5 stars,Sports Toys & Outdoor > Kites & Flight Toys,http://www.amazon.co.uk/3D-Prima-3DPPLA175BK |...,NaN,Technical Details Manufacturer recommended age...,NaN,http://www.amazon.co.uk/0-4mm-Drill-Printer-No...,NaN,Five Stars // 5.0 // 22 Sept. 2015 // By\n ...,NaN,<NA>
4250,fc63fb88dfae3d5ed32ba3cba3dd80e1,Disney Planes Fire & Rescue Deluxe Diecast Veh...,Disney,12.97,2.0,1.0,1.0,5.0 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Planes-Rescue-D...,Disney's Planes: Fire & Rescue Deluxe Diecast ...,Technical Details Item Weight159 g Product Dim...,Disney's Planes: Fire & Rescue Deluxe Diecast ...,http://www.amazon.co.uk/Disney-Planes-Rescue-D...,NaN,Five Stars // 5.0 // 19 Dec. 2015 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""ECM DIRECT"", ""S...",25.94
5794,69f2b4f0aabd0697c740e4a2faa5fc82,Italeri I8702 Model Locomotive BR50 Scale 1:87,Italeri,16.9,5.0,3.0,1.0,4.3 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Revell-87-Scale-Steam-...,Product Description Italeri 1/87/HO Lokomotive...,Technical Details Item Weight299 g Product Dim...,Product Description Italeri 1/87/HO Lokomotive...,http://www.amazon.co.uk/Revell-87-Scale-Steam-...,NaN,unusual locomotive // 4.0 // 1 Jan. 2016 // By...,NaN,84.5
3283,74afa3f2918b5280f438a26aea6f4197,Novelties Direct Prom King Satin Sash (1),Novelties Direct,5.23,<NA>,2.0,1.0,5.0 out of 5 stars,Fancy Dress > Costumes > Adults,http://www.amazon.co.uk/PROM-QUEEN-WHITE-SATIN...,Prom King Satin Sash. Make sure you remember t...,Technical Details Item Weight18 g Product Dime...,Prom King Satin Sash. Make sure you remember t...,http://www.amazon.co.uk/PROM-QUEEN-WHITE-SATIN...,NaN,Prom King Sash // 5.0 // 7 July 2013 // By\n ...,"{""seller""=>{""Seller_name_1""=>""Novelties Direct...",<NA>
7839,86e145cc6ff78035429fb7f51dacc9f2,Thomas & Friends Take-n-Play Ferdinand Engine,Thomas & Friends,7.35,23.0,7.0,1.0,4.9 out of 5 stars,Die-Cast & Toy Vehicles > Toy Trains & Accesso...,http://www.amazon.co.uk/Thomas-Friends-Take-n-...,Product Description Thomas and Friends Take-n-...,Technical Details Item Weight154 Kg Product Di...,Product Description Thomas and Friends Take-n-...,http://www.amazon.co.uk/Thomas-Friends-Take-n-...,NaN,Very happy with this item! // 5.0 // 3 Dec. 20...,"{""seller""=>[{""Seller_name_1""=>""a1 Toys"", ""Sell...",169.05
3580,b7d3a21394595bb7d2d02f1bca4144c2,Da Vinci Machines Series Model Kit - Armoured Car,Academy,12.99,3.0,1.0,1.0,5.0 out of 5 stars,Hobbies > Model Building Kits > Tanks,http://www.amazon.co.uk/Thumbs-Up-Da-Vinci-Cat...,Armoured Car - Da Vinci Machines Series Kit by...,Technical Details Item Weight286 g Product Dim...,Armoured Car - Da Vinci Machines Series Kit by...,http://www.amazon.co.uk/Thumbs-Up-Da-Vinci-Cat...,NaN,Five Stars // 5.0 // 10 Aug. 2015 // By\n \...,"{""seller""=>[{""Seller_name_1""=>""FACESHOP"", ""Sel...",38.97
3316,f1058a59fd43b0af7b908e0435823423,Childrens Roman Emperor fancy Dress Costume- M...,Bristol Novelties,6.54,20.0,38.0,2.0,4.5 out of 5 stars,NaN,http://www.amazon.co.uk/Tassel-Embellished | h...,Size Name:M Sizes 3-4 years; 5-7 years 8-10 ye...,Technical Details Product Dimensions35 x 35 x ...,Size Name:M Sizes 3-4 years; 5-7 years 8-10 ye...,http://www.amazon.co.uk/Rubies-Masquerade | ht...,NaN,Dress up // 5.0 // 6 Sept. 2012 // By\n \n ...,"{""seller""=>[{""Seller_name_1""=>""Theme Fancy Dre...",130.8
6984,2289b5990841b83829a0ca803f45f8fb,"DC Universe 6"" Classics Green Lantern Figure

In [31]:
df.isnull().sum() #product_information (TECH) has less missing vanues than offer part "product_description"

uniq_id                                           0
product_name                                      0
manufacturer                                      7
price_£                                        1453
new_in_stock                                   2500
number_of_reviews                                20
number_of_answered_questions                    765
average_review_rating                            18
amazon_category_and_sub_category                690
customers_who_bought_this_item_also_bought     1062
description                                     651
product_information                              58
product_description                             651
items_customers_buy_after_viewing_this_item    3065
customer_questions_and_answers                 9086
customer_reviews                                 21
sellers                                        3082
Total                                          3664
dtype: int64

In [38]:
# Replace '0' values with NaN for computation
df.replace(0, np.nan, inplace=True)

# Create separate DataFrames where 'product_information' and 'product_description' are NaN
df_info_nan = df[df['product_information'].isna()]
df_desc_nan = df[df['product_description'].isna()]

# Find common indices
common_indices = df_info_nan.index.intersection(df_desc_nan.index)

# Create a DataFrame with 2 common indices
df_common = df.loc[common_indices]

# Now, df_common is a DataFrame where both 'product_information' and 'product_description' are NaN

## EDA

In [42]:
# Fill NaN values with an empty string
df['product_description'] = df['product_description'].fillna('')

# Now your code should work
disney = df[df['product_description'].str.contains("disney")]

In [43]:
disney.head()

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
381,bdf9d6874160d07df78424271bdea6d7,Steiff Mickey Mouse,Steiff,119.95,12.0,1.0,1.0,5.0 out of 5 stars,NaN,http://www.amazon.co.uk/Steiff-354946-Minnie-M...,Steiff Mickey Mouse 24cmFor the lovers of disn...,Technical Details Item Weight739 g Product Dim...,Steiff Mickey Mouse 24cmFor the lovers of disn...,NaN,Does the item come boxed and with an official ...,Five Stars // 5.0 // 5 Aug. 2015 // By\n \n...,"{""seller""=>[{""Seller_name_1""=>""Bears to Collec...",1439.4
393,bbcca2bfe3b2eee013552a4b9ae33e10,Disney Mickey 2-in-1 Cushion Plush,Disney,8.05,4.0,8.0,1.0,4.1 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Disney-Minnie-2-in-1-C...,"Product Description Whether at home, on the ro...",Technical Details Item Weight200 g Product Dim...,"Product Description Whether at home, on the ro...",http://www.amazon.co.uk/Character-Disney-Micke...,NaN,Decent value for money. // 4.0 // 24 Nov. 2014...,"{""seller""=>[{""Seller_name_1""=>""PremierDropShop...",32.2
5016,c99175a322267bf2f155d151fe552500,Doc Mcstuffins Electronic Guitar,Disney Junior,10.56,6.0,9.0,1.0,4.2 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Doc-Mcstuffins | http:...,Product Description Have loads of fun with thi...,Technical Details Item Weight662 g Product Dim...,Product Description Have loads of fun with thi...,http://www.amazon.co.uk/Party-Bubbles-for-Favo...,NaN,Christmas favorite // 5.0 // 9 Feb. 2014 // By...,NaN,63.36
6816,30ec9bfa968b83b70459576006806b28,"Jakobs 4019999 - Schaukelwanne für Rody, circa...",Jakobs,15.0,4.0,4.0,2.0,5.0 out of 5 stars,Baby & Toddler Toys > Rockers & Ride-ons,http://www.amazon.co.uk/Gymnic-Inflatable | ht...,The gymnic line rody horse is a favorite with ...,Technical Details Item Weight1.3 Kg Product Di...,The gymnic line rody horse is a favorite with ...,http://www.amazon.co.uk/Dual-Action-Hand-Pump-...,NaN,Rody base for inflatable horse. // 5.0 // 19 J...,NaN,60.0
9375,9c660aa885696bea98a907033410c38b,Ravensburger Disney Cars 2 Memory Game,Ravensburger,9.99,7.0,8.0,1.0,4.6 out of 5 stars,Characters & Brands > Disney > Toys,http://www.amazon.co.uk/Ravensburger | http://...,"Product Description A matching pairs game, whi...",Technical Details Item Weight399 g Product Dim...,"Product Description A matching pairs game, whi...",http://www.amazon.co.uk/Ravensburger | http://...,NaN,Memory game // 4.0 // 9 Jan. 2012 // By\n \...,NaN,69.93


In [46]:
# Create a DataFrame with only 'Disney' products
df_disney = df[df['product_description'].str.contains('disney', case=False, na=False)]

# Convert to numerical value
df['average_review_rating'] = df['average_review_rating'].str.split().str[0].astype(float)

In [49]:
# Calculate the average rating for 'Disney'
df.head()

,uniq_id,product_name,manufacturer,price_£,new_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers,Total
0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,3.42,5.0,15.0,1.0,4.9,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ...",17.1
1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,16.99,<NA>,2.0,1.0,4.5,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ...",<NA>
2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,9.99,2.0,17.0,2.0,3.9,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel...",19.98
3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,39.99,<NA>,1.0,2.0,5.0,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN,<NA>
4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,32.19,<NA>,3.0,2.0,4.7,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN,<NA>


## Data Processing

In [51]:
# Convert 'average_review_rating' to numerical values
df_disney['average_review_rating'] = df_disney['average_review_rating'].str.split().str[0].astype(float)

<ipython-input-51-7947eb6264e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_disney['average_review_rating'] = df_disney['average_review_rating'].str.split().str[0].astype(float)


In [52]:
disney_avg = df_disney['average_review_rating'].mean()

In [54]:
# Convert 'average_review_rating' to numerical values using .loc
df_disney.loc[:, 'average_review_rating'] = df_disney['average_review_rating'].str.split().str[0].astype(float)

AttributeError: Can only use .str accessor with string values!

In [50]:
disney_avg = df_disney['average_review_rating'].mean()

TypeError: can only concatenate str (not "int") to str

In [ ]:

# Calculate the average rating for all brands
avg_rating_all_brands = df['average_review_rating'].mean()

print(f"Average rating for 'Disney': {disney_avg_rating}")
print(f"Average rating for all brands: {avg_rating_all_brands}")